<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">


# Building a groundwater model from scratch

This notebook shows how to build a basic model from scratch using `nlmod`.

In [ ]:
import flopy as fp
import pandas as pd

import nlmod

In [ ]:
nlmod.util.get_color_logger("INFO");

## Model parameters

In [ ]:
extent = [-500, 500, -500, 500]

top = 0.0
botm = [-10, -15, -30]

kh = [10, 0.1, 20]
kv = [0.5 * k for k in kh]

dx = 10.0
dy = 10.0

### Create model dataset

In [ ]:
ds = nlmod.get_ds(
    extent,
    delr=dx,
    delc=dy,
    top=top,
    botm=botm,
    kh=kh,
    kv=kv,
    model_ws="./scratch_model",
    model_name="from_scratch",
)
ds

### Set time discretisation

In [ ]:
ds = nlmod.time.set_ds_time(ds, time=pd.Timestamp.today(), start=1)

### Start building model

In [ ]:
sim = nlmod.sim.sim(ds)
tdis = nlmod.sim.tdis(ds, sim)
ims = nlmod.sim.ims(sim, complexity="SIMPLE")
gwf = nlmod.gwf.gwf(ds, sim)
dis = nlmod.gwf.dis(ds, gwf)
npf = nlmod.gwf.npf(ds, gwf)
ic = nlmod.gwf.ic(ds, gwf, starting_head=1.0)
oc = nlmod.gwf.oc(ds, gwf, save_head=True)

### Add wells

In [ ]:
wells = pd.DataFrame(columns=["x", "y", "top", "botm", "Q"], index=range(2))
wells.index.name = "well no."
wells.loc[0] = 100, -50, -5, -10, -100.0
wells.loc[1] = 200, 150, -20, -30, -300.0
wells

In [ ]:
wel = nlmod.gwf.wells.wel_from_df(wells, gwf)

### Add river

In [ ]:
xyriv = [
    (250, -500),
    (300, -300),
    (275, 0),
    (200, 250),
    (175, 500),
]

riv_layer = 0  # add to first layer

bed_resistance = 0.1  # days
riv_cond = dx * dy / bed_resistance  # conductance
riv_stage = 1.0  # m NAP
riv_botm = -3.0  # m NAP

In [ ]:
riv_data = nlmod.gwf.surface_water.rivdata_from_xylist(
    gwf, xyriv, riv_layer, riv_stage, riv_cond, riv_botm
)

riv = fp.mf6.ModflowGwfriv(gwf, stress_period_data={0: riv_data})

### Write and run Simulation

In [ ]:
nlmod.sim.write_and_run(sim, ds, silent=True)

### Load heads

In [ ]:
head = nlmod.gwf.get_heads_da(ds)

### Plot heads

In [ ]:
# using nlmod plotting methods
ax = nlmod.plot.map_array(
    head.sel(layer=0).isel(time=0),
    ds,
    cmap="RdYlBu",
    colorbar_label="head [m NAP]",
    xlabel="x [km]",
    ylabel="y [km]",
    title="head first layer",
    plot_grid=True,
)

### Plot heads in all layers

In [ ]:
# using xarray plotting methods
fg = head.plot(
    x="x",
    y="y",
    col="layer",
    col_wrap=3,
    cmap="RdYlBu",
    subplot_kws={"aspect": "equal"},
    cbar_kwargs={"label": "head [m NAP]"},
)

for iax in fg.axs.flat:
    nlmod.plot.modelgrid(ds, ax=iax, alpha=0.5, lw=0.5)